In [13]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
import random
import numpy as np
import pandas as pd
import os
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False
if not IN_COLAB:
    os.chdir("..")
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Utility Functions

In [14]:
def make_mixed_classification(n_samples, n_features, n_categories):
    X,y = make_classification(n_samples=n_samples, n_features=n_features, random_state=42, n_informative=5)
    cat_cols = random.choices(list(range(X.shape[-1])),k=n_categories)
    num_cols = [i for i in range(X.shape[-1]) if i not in cat_cols]
    for col in cat_cols:
        X[:,col] = pd.qcut(X[:,col], q=4).codes.astype(int)
    col_names = [] 
    num_col_names=[]
    cat_col_names=[]
    for i in range(X.shape[-1]):
        if i in cat_cols:
            col_names.append(f"cat_col_{i}")
            cat_col_names.append(f"cat_col_{i}")
        if i in num_cols:
            col_names.append(f"num_col_{i}")
            num_col_names.append(f"num_col_{i}")
    X = pd.DataFrame(X, columns=col_names)
    y = pd.Series(y, name="target")
    data = X.join(y)
    return data, cat_col_names, num_col_names

def print_metrics(y_true, y_pred, tag):
    if isinstance(y_true, pd.DataFrame) or isinstance(y_true, pd.Series):
        y_true = y_true.values
    if isinstance(y_pred, pd.DataFrame) or isinstance(y_pred, pd.Series):
        y_pred = y_pred.values
    if y_true.ndim>1:
        y_true=y_true.ravel()
    if y_pred.ndim>1:
        y_pred=y_pred.ravel()
    val_acc = accuracy_score(y_true, y_pred)
    val_f1 = f1_score(y_true, y_pred)
    print(f"{tag} Acc: {val_acc} | {tag} F1: {val_f1}")

# Generate Synthetic Data 

First of all, let's create a synthetic data which is a mix of numerical and categorical features

In [15]:
data, cat_col_names, num_col_names = make_mixed_classification(n_samples=10000, n_features=20, n_categories=4)
train, test = train_test_split(data, random_state=42)
train, val = train_test_split(train, random_state=42)

# Importing the Library

In [16]:
from pytorch_tabular import TabularModel
from pytorch_tabular.models import CategoryEmbeddingModelConfig
from pytorch_tabular.config import DataConfig, OptimizerConfig, TrainerConfig, ExperimentConfig
from pytorch_tabular.models.common.heads import LinearHeadConfig

## Define the Configs

This is the most crucial step in the process. There are four configs that you need to provide(most of them have intelligent default values), which will drive the rest of the process.

* DataConfig - Define the target column names, categorical and numerical column names, any transformation you need to do, etc.
* ModelConfig - There is a specific config for each of the models. This determines which model we are going to train and also lets you define the hyperparameters of the model
* TrainerConfig - This let's you configure the training process by setting things like batch_size, epochs, early stopping, etc. The vast majority of parameters are directly borrowed from PyTorch Lightning and is passed to the underlying Trainer object during training
* OptimizerConfig - This let's you define and use different Optimizers and LearningRate Schedulers. Standard PyTorch Optimizers and Learning RateSchedulers are supported. For custom optimizers, you can use the parameter in the fit method to overwrite this. The custom optimizer should be PyTorch compatible
* ExperimentConfig - This is an optional parameter. If set, this defines the Experiment Tracking. Right now, only two experiment tracking frameworks are supported: Tensorboard and Weights&Biases. W&B experiment tracker has more features like tracking the gradients and logits across epochs.

In [17]:
data_config = DataConfig(
    target=['target'], #target should always be a list. Multi-targets are only supported for regression. Multi-Task Classification is not implemented
    continuous_cols=num_col_names,
    categorical_cols=cat_col_names,
)
trainer_config = TrainerConfig(
    auto_lr_find=True, # Runs the LRFinder to automatically derive a learning rate
    batch_size=1024,
    max_epochs=100,
    accelerator="auto", # can be 'cpu','gpu', 'tpu', or 'ipu' 
    devices=-1 # -1 means use all available
)
optimizer_config = OptimizerConfig()

# DEPRECATED
# CategoryEmbeddingModel parameters like layers, activation etc. are now part of the head and head config
# Such parameters will be removed from the model config in the future
# model_config = CategoryEmbeddingModelConfig(
#     task="classification",
#     layers="1024-512-512",  # Number of nodes in each layer
#     activation="LeakyReLU", # Activation between each layers
#     learning_rate = 1e-3
# )

head_config = LinearHeadConfig(
    layers="32-16", # Number of nodes in each layer
    activation="LeakyReLU", # Activation between each layers
    dropout=0.1,
    initialization="kaiming"
).__dict__ # Convert to dict to pass to the model config (OmegaConf doesn't accept objects)

model_config = CategoryEmbeddingModelConfig(
    task="classification",
    head = "LinearHead", #Linear Head
    head_config = head_config, # Linear Head Config
    learning_rate = 1e-3
)


tabular_model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
)

## Training the Model 
Now that we have defined the configs and the TabularModel. We just need to call the `fit` method and pass the train and test dataframes. We can also pass in validation dataframe. But if omitted, TabularModel will separate 20%(also configurable) at random from the data as validation.

By default, EarlyStopping is enabled and is monitoring Validation Loss with a patience of 3 epochs. The trainer also saves the best model(based on validation loss) and loads that model at the end of training. `TrainerConfig` has the parameters to tweak this default behaviour.

In [18]:
tabular_model.fit(train=train, validation=val)

Global seed set to 42
/home/manujosephv/.virtualenvs/pytorch_env/lib/python3.8/site-packages/pytorch_lightning-1.5.9-py3.8.egg/pytorch_lightning/trainer/connectors/callback_connector.py:147: LightningDeprecationWarning: Setting `Trainer(checkpoint_callback=True)` is deprecated in v1.5 and will be removed in v1.7. Please consider using `Trainer(enable_checkpointing=True)`.
  rank_zero_deprecation(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/manujosephv/.virtualenvs/pytorch_env/lib/python3.8/site-packages/pytorch_lightning-1.5.9-py3.8.egg/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
Global seed set

Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

Restoring states from the checkpoint path at /home/manujosephv/pytorch_tabular/pytorch_tabular/lr_find_temp_model_4cb73040-e83a-4b8f-a181-b300048cb17c.ckpt
Learning rate set to 0.006918309709189364
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type                      | Params
---------------------------------------------------------------
0 | _backbone        | CategoryEmbeddingBackbone | 14.0 K
1 | _embedding_layer | Embedding1dLayer          | 92    
2 | head             | LinearHead                | 1.6 K 
3 | loss             | CrossEntropyLoss          | 0     
---------------------------------------------------------------
15.8 K    Trainable params
0         Non-trainable params
15.8 K    Total params
0.063     Total estimated model params size (MB)
/home/manujosephv/.virtualenvs/pytorch_env/lib/python3.8/site-packages/pytorch_lightning-1.5.9-py3.8.egg/pytorch_lightning/callbacks/model_checkpoint.py:631: UserWarning: Checkpoint directory /home/manujosephv/p

Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42


...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

## Evaluating the Model

### Loss and Metrics on New Data
To evaluate the model on new data on the same metrics/loss that was used during training, we can use the `evaluate` method

In [19]:
result = tabular_model.evaluate(test)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/manujosephv/.virtualenvs/pytorch_env/lib/python3.8/site-packages/pytorch_lightning-1.5.9-py3.8.egg/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_accuracy': 0.8407999873161316, 'test_loss': 0.3509310781955719}
--------------------------------------------------------------------------------


## New Predictions as DataFrame
To get the prediction as a dataframe, we can use the `predict` method. This will add predictions to the same dataframe that was passed in. For classification problems, we get both the probabilities and the final prediction taking 0.5 as the threshold

In [20]:
pred_df = tabular_model.predict(test)
pred_df.head()

Generating Predictions...:   0%|          | 0/3 [00:00<?, ?it/s]

,cat_col_0,num_col_1,num_col_2,num_col_3,cat_col_4,cat_col_5,num_col_6,num_col_7,num_col_8,num_col_9,...,num_col_14,num_col_15,num_col_16,num_col_17,num_col_18,num_col_19,target,0_probability,1_probability,prediction
6252,0.0,-3.304646,-2.010758,3.205420,1.0,1.0,0.427836,-1.492040,-1.061102,1.364186,...,-0.660336,-0.705788,0.229519,0.060878,-0.464394,2.879481,0,0.095677,0.904324,1
4684,1.0,-1.360640,-1.207160,2.690514,3.0,0.0,1.561682,0.953991,0.439317,1.243788,...,-2.726836,0.944248,0.821184,0.368647,-1.199147,0.126323,1,0.361086,0.638914,1
1731,1.0,-0.046718,-0.279572,0.363639,3.0,2.0,0.084824,0.194984,-1.005871,2.668561,...,-0.508633,0.508788,-0.097083,-0.128070,-0.282642,-0.190155,0,0.803429,0.196571,0
4742,1.0,1.549718,0.798527,0.916448,0.0,2.0,-0.903704,1.538725,0.475361,1.518521,...,0.326685,1.343219,-1.147619,1.795053,0.857619,0.532915,1,0.436286,0.563714,1
4521,2.0,0.909692,-0.117476,-0.168583,1.0,2.0,-1.233511,-0.137569,1.253686,-1.678887,...,-0.282845,0.458761,1.381926,-0.566849,-0.475947,-0.400418,1,0.118359,0.881641,1


In [21]:
print_metrics(test['target'], pred_df["prediction"], tag="Holdout")

Holdout Acc: 0.8408 | Holdout F1: 0.8579586009992862


## Saving and Loading the Model

In [22]:
tabular_model.save_model("examples/basic")

In [23]:
loaded_model = TabularModel.load_from_checkpoint("examples/basic")

/home/manujosephv/.virtualenvs/pytorch_env/lib/python3.8/site-packages/pytorch_lightning-1.5.9-py3.8.egg/pytorch_lightning/trainer/connectors/callback_connector.py:147: LightningDeprecationWarning: Setting `Trainer(checkpoint_callback=True)` is deprecated in v1.5 and will be removed in v1.7. Please consider using `Trainer(enable_checkpointing=True)`.
  rank_zero_deprecation(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [24]:
result = loaded_model.evaluate(test)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Output()

--------------------------------------------------------------------------------

DATALOADER:0 TEST RESULTS

{'test_accuracy': 0.8407999873161316, 'test_loss': 0.3509310781955719}

--------------------------------------------------------------------------------

/home/manujosephv/.virtualenvs/pytorch_env/lib/python3.8/site-packages/pytorch_lightning-1.5.9-py3.8.egg/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
